In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import json
import random
from tqdm import tqdm
from torchtune.modules import RotaryPositionalEmbeddings

d:\CourseworkFolder\DPSynthData\LLM PRAC\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 16
#block_size = 128
max_iters = 6000
eval_interval = 100
learning_rate = 6e-4
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [3]:
train_data = torch.load("train.pt")
val_data = torch.load("val.pt")

def find_max_len(dataset, name="Dataset"):
    max_len = 0
    total = 0
    count = 0
    for seq in dataset:
        seq_len = len(seq)
        if seq_len > max_len:
            max_len = seq_len
        total += seq_len
        count += 1
    avg_len = total / count if count > 0 else 0
    print(f"{name}: {count} timelines. Max length: {max_len}, Average length: {avg_len:.1f}")

find_max_len(train_data, "Train set")
find_max_len(val_data, "Val set")

with open("D:/CourseworkFolder/DPSynthData/Data Manipulation/token_map.json", "r", encoding="utf-8") as f:
    token_map = json.load(f)
vocab_size = len(token_map)
print(f"Vocab size: {vocab_size}")
 

Train set: 96377 timelines. Max length: 29941, Average length: 107.5
Val set: 10709 timelines. Max length: 27058, Average length: 109.7
Vocab size: 15567


In [4]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    batch = random.sample(data, batch_size)
    x_batch = [torch.tensor(seq[:-1], dtype=torch.long, device=device) for seq in batch]
    y_batch = [torch.tensor(seq[1:], dtype=torch.long, device=device) for seq in batch]
    return x_batch, y_batch

@torch.no_grad()

# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             x_batch, y_batch = get_batch(split)
            
#             batch_losses = []
#             for x, y in zip(x_batch, y_batch):
#                 logits, loss = model([x], [y])  
#                 batch_losses.append(loss.item())
            
#             avg_loss = sum(batch_losses) / len(batch_losses)
#             losses[k] = avg_loss
        
#         out[split] = losses.mean()
#     model.train()
#     return out

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, n_embd):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = n_embd // num_heads
        assert n_embd % num_heads == 0, "Embedding size must be divisible by num_heads"

        self.query = nn.Linear(n_embd, n_embd, bias=False)
        self.key = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)
        
        
        self.rope = RotaryPositionalEmbeddings(dim=self.head_dim, max_seq_len=50000)
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape  # (batch_size, sequence_length, embedding_dim)
        
        # Linear projections and reshape to (B, T, n_head, head_dim)
        q = self.query(x).view(B, T, self.num_heads, self.head_dim)
        k = self.key(x).view(B, T, self.num_heads, self.head_dim)
        v = self.value(x).view(B, T, self.num_heads, self.head_dim)
        
        #print(f"q.shape: {q.shape}")
        
        # Apply RoPE to q and k (automatic positional embeddings)
        q = self.rope(q)
        k = self.rope(k)

        # Scaled Dot-Product Attention with flash attention
        out = F.scaled_dot_product_attention(
            q, k, v,
            attn_mask=None,
            dropout_p=dropout if self.training else 0.0,
            is_causal=True
        )  # (B, T, n_head, head_dim)

        # Merge heads: (B, T, n_head, head_dim) -> (B, T, C)
        out = out.transpose(1, 2).reshape(B, T, C)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.sa = MultiHeadAttention(n_head, n_embd)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class TimelineLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        #self.position_embedding_table = nn.Embedding(12000, n_embd)  
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, x_batch, y_batch=None):
        logits_batch = []
        losses = []

        for x in x_batch:
            #T = x.size(0)
            tok_emb = self.token_embedding_table(x)
            #pos_emb = self.position_embedding_table(torch.arange(T, device=x.device))
            x_in = tok_emb 
            x_in = x_in.unsqueeze(0)

            x_out = self.blocks(x_in)
            x_out = self.ln_f(x_out)
            logits = self.lm_head(x_out).squeeze(0)  
            logits_batch.append(logits)

        if y_batch is not None:
            for logits, y in zip(logits_batch, y_batch):
                loss = F.cross_entropy(logits, y)
                losses.append(loss)
            return logits_batch, torch.stack(losses).mean()

        return logits_batch, None

    def generate(self, idx, max_new_tokens):
        min_tokens = 1
        eos_token = 15566

        for i in range(max_new_tokens):
            T = idx.size(1)
            tok_emb = self.token_embedding_table(idx)
            #pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
            x = tok_emb
            x = self.blocks(x)
            x = self.ln_f(x)
            logits = self.lm_head(x)

            probs = F.softmax(logits[:, -1, :], dim=-1)

            if i < min_tokens:
                probs[:, eos_token] = 0
                probs = probs / probs.sum(dim=-1, keepdim=True)

            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
         
            if next_token.item() == eos_token and i >= min_tokens:
                break

        return idx


In [6]:
print(" Starting training")
model = TimelineLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
print(f" Model has {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

for iter in tqdm(range(max_iters), desc="Training Model", unit="steps"):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f" Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)




 Starting training
 Model has 2.21M parameters


Training Model:   0%|          | 0/6000 [00:00<?, ?steps/s]

 Step 0: train loss 9.7831, val loss 9.7877


Training Model:   2%|▏         | 100/6000 [00:57<23:43,  4.14steps/s] 

 Step 100: train loss 6.1807, val loss 6.1662


...

Training Model:  97%|█████████▋| 5800/6000 [51:38<01:20,  2.48steps/s]

 Step 5800: train loss 3.1444, val loss 3.1744


Training Model:  98%|█████████▊| 5900/6000 [52:38<00:21,  4.71steps/s]

 Step 5900: train loss 3.1365, val loss 3.1423


Training Model: 100%|██████████| 6000/6000 [53:45<00:00,  1.86steps/s]


In [10]:
with open("D:/CourseworkFolder/DPSynthData/Data Manipulation/token_map.json", "r", encoding="utf-8") as f:
    token_map = json.load(f)
itos = {v: k for k, v in token_map.items()}

def decode(token_ids):
    return [itos.get(i, f"<unk:{i}>") for i in token_ids]


torch.save(model.state_dict(), "timeline_model.pt")
print(" Model saved.")

print(" Generating synthetic timeline...")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
output = model.generate(context, max_new_tokens=300)
decoded = decode(output[0].tolist())

print(" Decoded Tokens:")
for token in decoded:
    print(token)

 Model saved.
 Generating synthetic timeline...
 Decoded Tokens:
('Hospital Admission', 'admittime')_admission_location=AMBULATORY SURGERY TRANSFER_discharge_location=
Diagnosis_icd_code=Z20
Transfer to Location_careunit=Psychiatry
Transfer to Location_careunit=Psychiatry
Transfer to Location_careunit=Psychiatry
Transfer to Location_careunit=UNKNOWN
('Hospital Admission', 'admittime')_admission_location=PHYSICIAN REFERRAL_discharge_location=HOME
__EOS__
__EOS__
__EOS__
Transfer to Location_careunit=UNKNOWN
('Hospital Admission', 'admittime')_admission_location=PHYSICIAN REFERRAL_discharge_location=HOME
__EOS__
__EOS__
Transfer to Location_careunit=UNKNOWN
('Hospital Admission', 'admittime')_admission_location=EMERGENCY ROOM_discharge_location=HOME
__EOS__
__EOS__
Transfer to Location_careunit=UNKNOWN
('Hospital Admission', 'admittime')_admission_location=EMERGENCY ROOM_discharge_location=HOME
__EOS__
__EOS__
Transfer to Location_careunit=UNKNOWN
('Hospital Admission', 'admittime')_admi